In [1]:
pip install google-generativeai ollama openai

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import os
from dotenv import load_dotenv

# Cargar las variables desde el archivo .env
load_dotenv()

# Leer la variable del entorno
api_key = os.getenv("GOOGLE_API_KEY")

if not api_key:
    raise ValueError("No se encontró GOOGLE_API_KEY en el archivo .env")

print(f"API Key cargada correctamente: {api_key[:6]}... (oculta por seguridad)")

API Key cargada correctamente: AIzaSy... (oculta por seguridad)


In [4]:
import mlflow
import time
import random
import google.generativeai as genai
import requests

# Configurar Gemini
genai.configure(api_key=os.environ["GOOGLE_API_KEY"])

# Crear experimento
mlflow.set_experiment("laboratorio_3_llms")

# --- Función para simular costo (puedes ajustarla) ---
def estimate_cost(tokens, provider):
    if provider.lower() == "gemini":
        # Costos aproximados (USD) basados en tarifas reales de texto
        return tokens * 0.0000015
    elif provider.lower() == "ollama":
        # Asumimos costo casi nulo o simulado
        return tokens * 0.0000001
    return 0

# --- Función para ejecutar y registrar el modelo ---
def run_llm(provider, model_name, temperature, task_type, prompt):
    with mlflow.start_run(run_name=f"{provider}_run") as run:
        start = time.time()

        if provider.lower() == "gemini":
            model = genai.GenerativeModel(model_name)
            response = model.generate_content(prompt)
            text_response = response.text
            total_tokens = response.usage_metadata.total_token_count if hasattr(response, "usage_metadata") else len(prompt.split()) + len(text_response.split())
        elif provider.lower() == "ollama":
            # Llama al endpoint local de Ollama
            res = requests.post(
                "http://localhost:11434/api/generate",
                json={"model": model_name, "prompt": prompt, "stream": False}
            )
            response_json = res.json()
            text_response = response_json.get("response", "")
            total_tokens = len(prompt.split()) + len(text_response.split())
        else:
            raise ValueError("Proveedor no soportado")

        latency = time.time() - start
        cost = estimate_cost(total_tokens, provider)

        # Registrar parámetros
        mlflow.log_param("modelo", model_name)
        mlflow.log_param("proveedor", provider)
        mlflow.log_param("temperatura", temperature)
        mlflow.log_param("tipo_tarea", task_type)

        # Registrar métricas
        mlflow.log_metric("latencia", latency)
        mlflow.log_metric("total_tokens", total_tokens)
        mlflow.log_metric("costo_estimado", cost)

        # Guardar artefactos
        with open(f"prompt_{provider}.txt", "w", encoding="utf-8") as f:
            f.write(prompt)
        with open(f"respuesta_{provider}.txt", "w", encoding="utf-8") as f:
            f.write(text_response)

        mlflow.log_artifact(f"prompt_{provider}.txt")
        mlflow.log_artifact(f"respuesta_{provider}.txt")

        # Registrar modelo simbólico
        class DummyLLM(mlflow.pyfunc.PythonModel):
            def predict(self, context, model_input):
                return [text_response]

        model_name_registry = f"llm_{provider.lower()}_chat"
        model_info = mlflow.pyfunc.log_model(
            artifact_path="model",
            python_model=DummyLLM(),
            registered_model_name=model_name_registry
        )

        print(f"✅ {provider} registrado → {model_info.model_uri}")
        print(f"🕒 Latencia: {latency:.2f}s | Tokens: {total_tokens} | Costo: ${cost:.6f}")

# --- Ejecutar pruebas reales ---
prompt = "Explícame brevemente qué es el aprendizaje profundo en inteligencia artificial."
run_llm("Gemini", "gemini-2.5-flash", 0.7, "chat", prompt)
run_llm("Ollama", "llama3.1", 0.6, "chat", prompt)


2025/10/15 22:08:06 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'llm_gemini_chat'.
Created version '1' of model 'llm_gemini_chat'.


✅ Gemini registrado → runs:/70339e75db774ca38d7418d709c2a113/model
🕒 Latencia: 10.61s | Tokens: 1289 | Costo: $0.001934


2025/10/15 22:08:09 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


✅ Ollama registrado → runs:/eb7198f3fc704b698699dde0116b1087/model
🕒 Latencia: 2.04s | Tokens: 10 | Costo: $0.000001


Successfully registered model 'llm_ollama_chat'.
Created version '1' of model 'llm_ollama_chat'.
